# Hello 👋, <br>
This is my first Kaggle notebook. Any suggestions/ideas in the notebook are always welcome. <br>
I tried to keep this notebook as simple as possible. Using this notebook as a practise for my coding skills and tried describing it in a little funny way as well. 
# If you are a beginner then welcome and scroll down for a story!!!

# I want to explore the following topics in the following order
    
1. EDA with emphasis on India first then followed by all countries <br>
2. Simple linear regression <br>
3. Multiple Linear regression <br>
4. Finding the feature with least contribution to the model <br>
5. Finally develop an improved model by removing some features <br>
6. Compare Acuuracies of models with All features and Reduced Features <br>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***We are given 5 years of datasets. Let us consider only one year dataset since our intention of this notebook is to practise and understand regression***

In [ ]:
filenames

# 1. Exploratory Data Analysis (EDA)
**Let us explore dataset of year 2019 and see who is happy and who is not!**

In [ ]:
data_2019 = pd.read_csv('/kaggle/input/world-happiness/2019.csv')

**How many countries data is present and how many features are affecting individuals happiness** <br>
I want to see the full shape of data

In [ ]:
data_2019.shape             # data_2019 dataframe has a shape of 156 by 9.
                            # shape is an attribute of dataframe hence parenthesis is not followed 

In [ ]:
data_2019.head()           # head() method by default returns 5 examples. We can specify required number of examples inside parenthesis

**Whaaaaaaaat!!! India is not in top 5 countries** <br>
May be India is in Top 10 countries. Lets see (add fingers cross symbol)

In [ ]:
data_2019.head(10)

**May be in Top 50 countries**

In [ ]:
data_2019.head(50)

**Not even in top 50?? Definitely I was not counted during the survey 😜** <br>
Okay, but at which rank india actually is?

In [ ]:
rank_india = data_2019[data_2019["Country or region"]=="India"]["Overall rank"]   # This is called subsetting of a dataframe
print(rank_india)                               

In [ ]:
print("Yeaaah!! India is ahead of {} countries and only 139 to cross".format((data_2019.shape[0] - 140)))  # Just practising .format method
print("{} percent of the countries are happier than India".format(round(140/156,3)))

In [ ]:
data_2019.info()                # Checking if there are any null values

In [ ]:
data_2019.describe()      # Identifying max min and median for each feature       

In [ ]:
data_2019.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.pairplot(data_2019)

In [ ]:
sns.heatmap(data_2019.corr(),annot=True,cmap='RdYlGn',linewidths=0.2) #data.corr()-->correlation matrix
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

## Observation from GDP vs Perception of corruption.
* Higher GDP countries are likely to have higher corruption
* And interestingly highly corrupted countries are having high score
* lets plot GDP vs Perception of corruption separately

In [ ]:
plt.scatter(data_2019["GDP per capita"],data_2019["Perceptions of corruption"])
plt.show()

In [ ]:
GDP_per_capita = data_2019["GDP per capita"]
corruption = data_2019["Perceptions of corruption"]
Score = data_2019["Score"]

plt.style.use('ggplot')
plt_1 = plt.figure(figsize=(12, 5))

plt.subplot(1,2,1)
plt.scatter(GDP_per_capita, corruption)
plt.xlabel("GDP per capita")
plt.ylabel("Perception of corruption")

plt.subplot(1,2,2)
plt.scatter(GDP_per_capita, Score)
plt.xlabel("GDP per capita")
plt.ylabel("Score")
plt.show()

**Whichever countries have higher GDP per capita and perceptions of corruption they are most likely to be the happier countries(score high)**

# 2. Simple Linear Regression

**Analysing by going through each and every pairplot is definitely a tedious job. Sticking to the agenda of this notebook(run simple regression and multiple regression) lets start running a simple regression on Score vs GDP per capita.** <br>
Before actually performing regression let us refresh concepts with the below points <br>
- Regression means going to the lowest form or in reverse direction. Lets take a calculator, we input the numbers and it runs the calculations and gives the output because the calaculator already has the model(equation) to give output. Whereas in regression, we have input and output data, we need to find the best fitting model. Thus the word 'regress' best fits.
- Simple regression has only two variables (one input and one output)
- Multiple regression has more than two variables (more than one input and one output variable)<br>


***A linear relation between Score vs GDP per capita can be clearly seen in the above graph. Let's run regression on these two features ***

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [ ]:
GDP_per_capita = data_2019["GDP per capita"].values.reshape(-1,1)
Score = data_2019["Score"].values.reshape(-1,1)
print(type(GDP_per_capita))

In [ ]:
GDP_train, GDP_test, Score_train, Score_test = train_test_split(GDP_per_capita, Score, test_size = 0.3, random_state=42)

In [ ]:
# Checking how the data is splitted
plt.scatter(GDP_train,Score_train,c = "red")                    # plotting train data # learned how to give colour using 'c' argument
plt.scatter(GDP_test,Score_test,c = "green")                    # plotting test data
plt.xlabel("GDP per capita")
plt.ylabel("Score")
plt.show()

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(GDP_train, Score_train)

In [ ]:
intercept = reg.intercept_
print(intercept)                         # output is a list
coefficient= reg.coef_
print(coefficient)                       # output is a list of list. 

# Because generally there will be more than one input fetaures resulting in corresponding coefficients

print("Regression model : Score = {}*GDP per capita + {}".format(coefficient[0][0],intercept[0]))

In [ ]:
Score_test_predicted = reg.predict(GDP_test)
Score_train_predicted = reg.predict(GDP_train)
print(Score_test_predicted)

In [ ]:
from sklearn import metrics
metrics.mean_squared_error(Score_test,Score_test_predicted)
r2_score_test = metrics.r2_score(Score_test,Score_test_predicted)
r2_score_train = metrics.r2_score(Score_train,Score_train_predicted)
print("R2 score of test data is {}".format(r2_score_test))
print("R2 score of train data is {}".format(r2_score_train))

0.48 is too bad fit !!

In [ ]:
plt.scatter(GDP_test,Score_test,c = "red")                   # plotting train data
plt.plot(GDP_test,Score_test_predicted,c = "blue")                # This will be a staright line since it is a linear model
plt.xlabel("GDP per capita")
plt.ylabel("Score")
plt.show()

## Errors normally distributed?

In [ ]:
error = (Score_train - Score_train_predicted)
plt.hist(error, bins = 7)

# 3. Multiple Linear regression <br>
Its obvious that simple linear regression is not going to result in best fit for this dataset. Now we will try multiple regression.

**Regression is**
1. Outlier sensitive
2. Features need to be scaled to avoid biased weightage to features

In [ ]:
from sklearn import preprocessing             # scaling features to same scale

In [ ]:
print(data_2019.shape)
print(data_2019.columns)
data_2019.drop(["Country or region"], axis = 1, inplace = True)

In [ ]:
print(data_2019.shape)
columns_names = data_2019.columns
data_2019_norm = preprocessing.normalize(data_2019)
data_2019_norm

In [ ]:
df = pd.DataFrame(data_2019_norm, index = data_2019.index, columns = data_2019.columns)
df.head()

In [ ]:
df.describe()

Obervations. Not all features have come to 0 -1

In [ ]:
from sklearn import preprocessing
  
min_max_scaler = preprocessing.MinMaxScaler(feature_range =(0, 1))

data_2019_scaled= min_max_scaler.fit_transform(data_2019)
data_2019_scaled_df = pd.DataFrame(data_2019_scaled, index = data_2019.index, columns = data_2019.columns)

In [ ]:
print(data_2019_scaled_df.describe())

**Wow, Now, every feature is between 0 -1**

In [ ]:
X = data_2019_scaled_df.drop(["Score"], axis = 1)   # Since X is a matrix capital letter is used. This is a Convectional way.
y = data_2019_scaled_df["Score"]                    # Target variable is a vector, hence small letter y
X.head()

In [ ]:
from sklearn import linear_model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

In [ ]:
y_test_pred = reg.predict(X_test)
y_train_pred = reg.predict(X_train)

In [ ]:
r2_score_test = metrics.r2_score(y_test,y_test_pred)
r2_score_train = metrics.r2_score(y_train,y_train_pred)
print("R2 score of test data is {}".format(r2_score_test))
print("R2 score of train data is {}".format(r2_score_train))

In [ ]:
os.listdir()

In [ ]:
print(reg.fit(X_train, y_train).summary())

In [ ]:
params = np.append(reg.intercept_,reg.coef_)
predictions = reg.predict(X_train)

In [ ]:
print(predictions)